# 工厂规划 II

## 目标和前提条件

准备好迎接一个重大的生产规划挑战了吗？尝试这个示例，您将学习如何创建一个最优生产计划，该计划不仅可以实现利润最大化，还可以确定在哪个月份对机器进行维护操作。

关于这类模型的更多信息可以在 H. P. Williams 所著的《数学规划中的模型构建》第五版第 256 页和第 302-303 页的示例 #4 中找到。

这个建模示例属于中级水平，我们假设您了解 Python 并熟悉 Gurobi Python API。此外，您还应该具备一些构建数学优化模型的知识。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

一个工厂生产七种产品（Prod 1 到 Prod 7），使用多种机器，包括：

- 四台磨床
- 两台立式钻床
- 三台卧式钻床
- 一台镗床
- 一台刨床

每种产品都有确定的单位利润贡献（定义为单位销售价格减去原材料成本）。此外，每种产品的制造需要在每台机器上占用特定的时间（以小时计）。利润和制造时间值如下所示。横线表示该产品的制造过程不需要该机器。

| <i></i> | PROD1 | PROD2 | PROD3 | PROD4 | PROD5 | PROD6 | PROD7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 利润 | 10 | 6 | 8 | 4 | 11 | 9 | 3 |
| 磨床 | 0.5 | 0.7 | - | - | 0.3 | 0.2 | 0.5 |
| 立式钻床 | 0.1 | 0.2 | - | 0.3 | - | 0.6 | - |
| 卧式钻床 | 0.2 | - | 0.8 | - | - | - | 0.6 |
| 镗床 | 0.05 | 0.03 | - | 0.07 | 0.1 | - | 0.08 |
| 刨床 | - | - | 0.01 | - | 0.05 | - | 0.05 |

与工厂规划 I 不同，在这个版本的模型中，我们不预先定义机器的维护计划，而是将维护计划也作为优化目标。

维护要求如下：

- 每台机器必须在六个月中的某一个月进行维护停机。
- 上述规定的例外是磨床，在六个月期间只需要两台磨床进行维护。

每个月每种产品的销售量都有限制。这些限制如下所示：

| 月份 | PROD1 | PROD2 | PROD3 | PROD4 | PROD5 | PROD6 | PROD7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 一月 | 500 | 1000 | 300 | 300 | 800 | 200 | 100 |
| 二月 | 600 | 500 | 200 | 0 | 400 | 300 | 150 |
| 三月 | 300 | 600 | 0 | 0 | 500 | 400 | 100 |
| 四月 | 200 | 300 | 400 | 500 | 200 | 0 | 100 |
| 五月 | 0 | 100 | 500 | 100 | 1000 | 300 | 0 |
| 六月 | 500 | 500 | 100 | 300 | 1100 | 500 | 60 |

每种产品最多可以储存 100 个单位的库存，每单位每月的库存成本为 0.50 美元。一月初时没有产品库存。但到六月底时，每种产品都应有 50 个单位的库存。

工厂每周工作六天，每天两个八小时班次。可以假设每月有 24 个工作日。此外，就本模型而言，不需要考虑生产排序问题。

生产和维护计划应该是什么样的？同时，是否可以建议任何价格上涨，并确定获取新机器的价值？

这个问题基于为康沃尔工程公司 Holman Brothers 建立的一个更大规模的模型。

---
## 模型构建

### 集合和索引

$t \in \text{Months}=\{\text{Jan},\text{Feb},\text{Mar},\text{Apr},\text{May},\text{Jun}\}$：月份集合。

$p \in \text{Products}=\{1,2,\dots,7\}$：产品集合。

$m \in \text{Machines}=\{\text{Grinder},\text{VertDrill},\text{horiDrill},\text{Borer},\text{Planer}\}$：机器集合。

### 参数

$\text{hours_per_month} \in \mathbb{R}^+$：每月任何机器可用的时间（小时/月）。它是工作日数（24天）乘以每天的班次数（2）再乘以每班的时长（8小时）的结果。

$\text{max_inventory} \in \mathbb{N}$：任何给定月份中单一产品类型可存储的最大数量。

$\text{holding_cost} \in \mathbb{R}^+$：任何产品类型的月度库存成本（美元/单位/月）。

$\text{store_target} \in \mathbb{N}$：规划期末每种产品类型需保持的库存数量。

$\text{profit}_p \in \mathbb{R}^+$：产品 $p$ 的利润（美元/单位）。

$\text{installed}_m \in \mathbb{N}$：工厂安装的 $m$ 类型机器数量。

$\text{down_req}_{m} \in \mathbb{N}$：$m$ 类型机器在规划期内需要安排维护的数量。

$\text{time_req}_{m,p} \in \mathbb{R}^+$：在机器 $m$ 上制造一个单位产品 $p$ 所需的时间（小时/单位）。

$\text{max_sales}_{t,p} \in \mathbb{N}$：在月份 $t$ 中产品 $p$ 的最大销售数量。

### 决策变量

$\text{make}_{t,p} \in \mathbb{R}^+$：在月份 $t$ 制造产品 $p$ 的数量。

$\text{store}_{t,p} \in [0, \text{max_inventory}] \subset \mathbb{R}^+$：在月份 $t$ 储存产品 $p$ 的数量。

$\text{sell}_{t,p} \in [0, \text{max_sales}_{t,p}] \subset \mathbb{R}^+$：在月份 $t$ 销售产品 $p$ 的数量。

$\text{repair}_{t,m} \in \{0,1,\dots, \text{down_req}_m\} \subset \mathbb{N}$：在月份 $t$ 安排维护的 $m$ 类型机器数量。

**假设：** 我们可以生产分数单位。

### 目标函数

- **利润：** 最大化规划期内的总利润（美元）。

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\sum_{p \in \text{Products}}
(\text{profit}_p*\text{make}_{t,p} - \text{holding_cost}*\text{store}_{t,p})
\tag{0}
\end{equation}

### 约束条件

- **初始平衡：** 对于每个产品 $p$，生产的单位数量应等于销售的单位数量加上储存的单位数量（以产品单位计）。

\begin{equation}
\text{make}_{\text{Jan},p} = \text{sell}_{\text{Jan},p} + \text{store}_{\text{Jan},p} \quad \forall p \in \text{Products}
\tag{1}
\end{equation}

- **平衡：** 对于每个产品 $p$，月份 $t$ 生产的单位数量和之前储存的单位数量应等于该月销售和储存的单位数量（以产品单位计）。

\begin{equation}
\text{store}_{t-1,p} + \text{make}_{t,p} = \text{sell}_{t,p} + \text{store}_{t,p} \quad \forall (t,p) \in \text{Months} \setminus \{\text{Jan}\} \times \text{Products}
\tag{2}
\end{equation}

- **库存目标：** 规划期末每种产品 $p$ 的库存数量应达到目标（以产品单位计）。

\begin{equation}
\text{store}_{\text{Jun},p} = \text{store_target} \quad \forall p \in \text{Products}
\tag{3}
\end{equation}

- **机器产能：** 任何产品在机器类型 $m$ 上的总制造时间不能超过该月该机器的可用小时数（以小时计）。

\begin{equation}
\sum_{p \in \text{Products}}\text{time_req}_{m,p}*\text{make}_{t,p} \leq \text{hours_per_month}*(\text{installed}_m - \text{repair}_{t,m}) \quad \forall (t,m) \in \text{Months} \times \text{Machines}
\tag{4}
\end{equation}

- **维护：** 安排维护的 $m$ 类型机器数量应满足要求。

\begin{equation}
\sum_{t \in \text{Months}}\text{repair}_{t,m} = \text{down_req}_m \quad \forall m \in \text{Machines}
\tag{5}
\end{equation}

---
## Python 实现

我们导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
# %pip install gurobipy

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据
我们定义模型的所有输入数据。

In [2]:
# 参数定义

products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

profit = {"Prod1":10, "Prod2":6, "Prod3":8, "Prod4":4, "Prod5":11, "Prod6":9, "Prod7":3}

time_req = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}

# 每种机器的可用数量
installed = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

# 需要进行维护的机器数量
down_req = {"grinder":2, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

# 销售的市场限制
max_sales = {
    ("Jan", "Prod1") : 500,
    ("Jan", "Prod2") : 1000,
    ("Jan", "Prod3") : 300,
    ("Jan", "Prod4") : 300,
    ("Jan", "Prod5") : 800,
    ("Jan", "Prod6") : 200,
    ("Jan", "Prod7") : 100,
    ("Feb", "Prod1") : 600,
    ("Feb", "Prod2") : 500,
    ("Feb", "Prod3") : 200,
    ("Feb", "Prod4") : 0,
    ("Feb", "Prod5") : 400,
    ("Feb", "Prod6") : 300,
    ("Feb", "Prod7") : 150,
    ("Mar", "Prod1") : 300,
    ("Mar", "Prod2") : 600,
    ("Mar", "Prod3") : 0,
    ("Mar", "Prod4") : 0,
    ("Mar", "Prod5") : 500,
    ("Mar", "Prod6") : 400,
    ("Mar", "Prod7") : 100,
    ("Apr", "Prod1") : 200,
    ("Apr", "Prod2") : 300,
    ("Apr", "Prod3") : 400,
    ("Apr", "Prod4") : 500,
    ("Apr", "Prod5") : 200,
    ("Apr", "Prod6") : 0,
    ("Apr", "Prod7") : 100,
    ("May", "Prod1") : 0,
    ("May", "Prod2") : 100,
    ("May", "Prod3") : 500,
    ("May", "Prod4") : 100,
    ("May", "Prod5") : 1000,
    ("May", "Prod6") : 300,
    ("May", "Prod7") : 0,
    ("Jun", "Prod1") : 500,
    ("Jun", "Prod2") : 500,
    ("Jun", "Prod3") : 100,
    ("Jun", "Prod4") : 300,
    ("Jun", "Prod5") : 1100,
    ("Jun", "Prod6") : 500,
    ("Jun", "Prod7") : 60,
}

holding_cost = 0.5
max_inventory = 100
store_target = 50
hours_per_month = 2*8*24

## 模型部署
我们创建一个模型和变量。我们将 UpdateMode 参数设置为 1（这简化了代码 - 详见文档）。对于每种产品（七种产品）和每个时间段（月份），我们将创建变量来表示将生产、储存和销售的产品数量。每月每种产品的销售量都有上限。这是由于市场限制。对于每种类型的机器和每个月份，我们创建一个变量 d，用于表示该月该类型机器停机的数量。

In [3]:
factory = gp.Model('factory planning II')

make = factory.addVars(months, products, name="Make") # 生产数量
store = factory.addVars(months, products, ub=max_inventory, name="Store") # 储存数量
sell = factory.addVars(months, products, ub=max_sales, name="Sell") # 销售数量
repair = factory.addVars(months, machines, vtype=GRB.INTEGER, ub=down_req, name="Repair") # 停机维护的机器数量

Set parameter LicenseID to value 2601452


接下来，我们插入约束条件。
平衡约束确保上月储存的产品数量加上本月生产的数量等于本月销售和储存的数量，这适用于每种产品。这确保了模型中的所有产品都在某个月份生产。初始库存为空。

In [4]:
#1. 初始平衡
Balance0 = factory.addConstrs((make[months[0], product] == sell[months[0], product] 
                  + store[months[0], product] for product in products), name="Initial_Balance")
    
#2. 平衡
Balance = factory.addConstrs((store[months[months.index(month) -1], product] + 
                make[month, product] == sell[month, product] + store[month, product] 
                for product in products for month in months 
                if month != months[0]), name="Balance")

最终库存约束强制要求在最后一个月结束时每种产品的库存达到指定数量。

In [5]:
#3. 库存目标
TargetInv = factory.addConstrs((store[months[-1], product] == store_target for product in products),  name="End_Balance")

产能约束确保每个月所有产品在某种机器上需要的时间小于或等于该月该机器的可用小时数乘以该月可用机器数量。每种产品在不同机器上都需要一定的机器小时数。由于维护原因，每台机器在一个或多个月份都会停机，因此每月可用的机器数量和类型会有所变化。每种机器类型可能有多台机器。

In [6]:
#4. 机器产能
MachineCap = factory.addConstrs((gp.quicksum(time_req[machine][product] * make[month, product]
                             for product in time_req[machine])
                    <= hours_per_month * (installed[machine] - repair[month, machine])
                    for machine in machines for month in months),
                   name = "Capacity")

维护约束确保在某个月份内停机维护的机器数量和类型符合要求。现在，哪个月停机维护是优化的一部分。

In [7]:
#5. 维护

Maintenance = factory.addConstrs((repair.sum('*', machine) == down_req[machine] for machine in machines), "Maintenance")

目标是最大化公司的利润，这包括每种产品的利润减去未售出产品的库存成本。可以表示为：

In [8]:
#0. 目标函数
obj = gp.quicksum(profit[product] * sell[month, product] -  holding_cost * store[month, product]  
               for month in months for product in products)

factory.setObjective(obj, GRB.MAXIMIZE)

接下来，我们开始优化，Gurobi 找到最优解。

In [9]:
factory.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 84 rows, 156 columns and 348 nonzeros
Model fingerprint: 0x861f9886
Variable types: 126 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+02]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -175.0000000
Presolve removed 22 rows and 27 columns
Presolve time: 0.01s
Presolved: 62 rows, 129 columns, 278 nonzeros
Variable types: 105 continuous, 24 integer (12 binary)

Root relaxation: objective 1.164550e+05, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

---
## 分析

优化模型的结果显示，我们可以实现的最大利润为 $\$108,855.00$。与工厂规划 I 示例相比，在六个月内增加了 $\$15,139.82$，这是因为可以选择维护计划而不是固定的维护计划。让我们看看实现该最优结果的解决方案。

### 生产计划
该计划确定了在规划期内每个时期生产的每种产品的数量。例如，在二月份我们生产 600 个单位的产品 Prod1。

In [10]:
rows = months.copy()
columns = products.copy()
make_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in make.keys():
    if (abs(make[month, product].x) > 1e-6):
        make_plan.loc[month, product] = np.round(make[month, product].x, 1)
make_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,1000.0,300.0,300.0,800.0,200.0,100.0
Feb,600.0,500.0,200.0,0.0,400.0,300.0,150.0
Mar,400.0,700.0,100.0,100.0,600.0,400.0,200.0
Apr,0.0,0.0,0.0,0.0,0.0,0.0,0.0
May,0.0,100.0,500.0,100.0,1000.0,300.0,0.0
Jun,550.0,550.0,150.0,350.0,1150.0,550.0,110.0


### 销售计划
该计划定义了在规划期内每个时期销售的每种产品的数量。例如，在二月份我们销售 600 个单位的产品 Prod1。

In [11]:
rows = months.copy()
columns = products.copy()
sell_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in sell.keys():
    if (abs(sell[month, product].x) > 1e-6):
        sell_plan.loc[month, product] = np.round(sell[month, product].x, 1)
sell_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,1000.0,300.0,300.0,800.0,200.0,100.0
Feb,600.0,500.0,200.0,0.0,400.0,300.0,150.0
Mar,300.0,600.0,0.0,0.0,500.0,400.0,100.0
Apr,100.0,100.0,100.0,100.0,100.0,0.0,100.0
May,0.0,100.0,500.0,100.0,1000.0,300.0,0.0
Jun,500.0,500.0,100.0,300.0,1100.0,500.0,60.0


### 库存计划
该计划反映了在规划期内每个时期末的产品库存数量。例如，在二月底我们有 0 个单位的 Prod1 库存。

In [12]:
rows = months.copy()
columns = products.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in store.keys():
    if (abs(store[month, product].x) > 1e-6):
        store_plan.loc[month, product] = np.round(store[month, product].x, 1)
store_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Feb,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mar,100.0,100.0,100.0,100.0,100.0,0.0,100.0
Apr,0.0,0.0,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jun,50.0,50.0,50.0,50.0,50.0,50.0,50.0


### 维护计划
此计划显示了规划期内每个时段的维护计划。例如，四月份将有 2 台磨床进行维护停机。

In [13]:
rows = months.copy()
columns = machines.copy()
repair_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, machine in repair.keys():
    if (abs(repair[month, machine].x) > 1e-6):
        repair_plan.loc[month, machine] = repair[month, machine].x
repair_plan

,grinder,vertDrill,horiDrill,borer,planer
Jan,0.0,0.0,1.0,0.0,0.0
Feb,0.0,1.0,0.0,0.0,0.0
Mar,0.0,0.0,0.0,0.0,0.0
Apr,2.0,0.0,0.0,1.0,1.0
May,0.0,1.0,0.0,0.0,0.0
Jun,0.0,0.0,2.0,0.0,0.0


**注意：** 如果您想将解决方案写入文件，而不是打印到终端，可以使用 model.write() 命令。示例实现如下：

`factory.write("factory-planning-2-output.sol")`

---
## 参考文献

H. Paul Williams，《数学规划中的模型构建》，第五版。

Copyright © 2020 Gurobi Optimization, LLC